# Backed by a Vector Store
---

VectorStoreRetrieverMemory almacena recuerdos en un almacén de vectores y consulta los documentos más "destacados" cada vez que se llama.

Esto difiere de la mayoría de las otras clases de Memoria en que no rastrea explícitamente el orden de las interacciones.

En este caso, los "documentos" son fragmentos de conversaciones anteriores. Esto puede resultar útil para hacer referencia a información relevante que se le proporcionó a la IA anteriormente en la conversación.

In [2]:
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from langchain.chains import ConversationChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.prompts import PromptTemplate

load_dotenv(find_dotenv())

True

### Inicializamos la vector store

Dependiendo de la tienda que elijas, este paso puede verse diferente. Consulte la documentación relevante de la tienda de vectores para obtener más detalles.

In [4]:
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import Chroma

persist_directory = '../chromadb/memory-store/'

embeddings = OpenAIEmbeddings()
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_directory
)
vectorstore.persist()

### Creando tu VectorStoreRetrieverMemory

El objeto de memoria se crea una instancia desde cualquier recuperador de almacén de vectores.


In [7]:
# En el uso real, establecerías `k` como un valor más alto, pero usamos k=1 para mostrar que
# la búsqueda de vectores aún devuelve la información semánticamente relevante
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #

In [8]:
# Observe que el primer resultado devuelto es el recuerdo relacionado con la ayuda fiscal, que el modelo de lenguaje considera más semánticamente relevante.
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

input: My favorite sport is soccer
output: ...


## Uso en un "chain"

Veamos un ejemplo, configurando nuevamente `verbose=True` para que podamos ver el mensaje.

In [9]:
# llm
llm = OpenAI(temperature=0)

_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"],
    template=_DEFAULT_TEMPLATE
)

conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True,
)

In [10]:
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


" Hi Perry, I'm doing great. How about you?"

In [11]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="what's my favorite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


' You told me earlier that your favorite sport is soccer.'

In [12]:
# Even though the language model is stateless, since relevant memory is fetched, it can "reason" about the time.
# Timestamping memories and data is useful in general to let the agent determine temporal relevance
conversation_with_summary.predict(input="Whats my favorite food")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite food
AI:

> Finished chain.


' You said your favorite food is pizza.'

In [13]:
# The memories from the conversation are automatically stored,
# since this query best matches the introduction chat above,
# the agent is able to 'remember' the user's name.
conversation_with_summary.predict(input="What's my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is Perry, what's up?
response:  Hi Perry, I'm doing great. How about you?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What's my name?
AI:

> Finished chain.


' Your name is Perry.'